# Kyber algorithm

## Description

This algorithm uses to provide security KEM in TLS, encryption, IoT etc, OpenSSL.

## Math Problem

### Lattice-problem

The lattice problem is a class of mathematical problems related to the geometric structure of lattices, which are grids of regularly spaced points in 
n
n-dimensional space. Key problems include the Shortest Vector Problem (SVP) and the Closest Vector Problem (CVP). SVP seeks the shortest non-zero vector in a lattice, while CVP finds the lattice point nearest to a given target. These problems are foundational in cryptography, especially in constructing post-quantum cryptographic schemes due to their computational hardness. Lattice problems are challenging even for quantum computers, making them a cornerstone for secure communication in the face of advancing quantum technology.

## Functions

n = ? - param of kyber can be different<br>
q = ? - param of kyber can be different<br>
m = x^n + x^(n-1) ... x^0 - message

### Keygen

Key generation working has a few steps:

1. It's a determination of **s**-polynom and **e**-polynom by normal with mod **q**
2. Find **a** like a random polynom 0-q 
3. Find b as (a*s + e) mod q

s - private key<br>
(a, b) - public key

### Encrypt

Encryption has a few steps:

1. r - bit's polynom
2. Binomial gen of e1 and e2
3. find c1 and c2 <br>
c1 = (a*r + e1) mod q<br>
c2 = (b*r + e2 + (m*(q/2)) mod q) mod q

### Decrypt

Decryption example with math termin:

1. m1 = (c2 - s * c1) mod q
2. m = round(2 * m1 / q)% 2


In [42]:
import numpy as np

In [43]:
q_conf = 7
n_conf = 3
m_conf = np.random.randint(0, 2, size=n_conf)  # Binary message

In [44]:
def keygen(q, n):
    a = np.random.randint(0, q, size=n)
    s = np.random.normal(loc=0, scale=1, size=n).astype(int) % q
    e = np.random.normal(loc=0, scale=1, size=n).astype(int) % q
    b = (a * s + e) % q
    return s, (a, b), e

In [45]:
def encrypt(m, pub_key, n, q):
    r = np.random.randint(0, 2, size=n)
    e1 = np.random.binomial(1, 0.5, size=n) - np.random.binomial(1, 0.5, size=n)
    e2 = np.random.binomial(1, 0.5, size=n) - np.random.binomial(1, 0.5, size=n)
    c1 = (pub_key[0] * r + e1) % q
    c2 = (pub_key[1] * r + e2 + (m * (q // 2)) % q) % q
    return c1, c2

In [46]:
def decrypt(c, priv_key, n, q):
    c1, c2 = c
    decrypted = (c2 - priv_key * c1) % q
    m = (2 * decrypted / q).round() % 2
    return m.astype(int)

In [47]:
priv_key, pub_key, error = keygen(q_conf, n_conf)
c1, c2 = encrypt(m_conf, pub_key, n_conf, q_conf)
m = decrypt((c1, c2), priv_key, n_conf, q_conf)

In [48]:
print("Original Message:", m_conf)
print("-" * 30)
print("Private Key:", priv_key)
print("Public Key (a, b):", pub_key)
print("-" * 30)
print("Ciphertext c1:", c1)
print("Ciphertext c2:", c2)
print("-" * 30)
print("Decrypted Message:", m)

Original Message: [0 1 1]
------------------------------
Private Key: [0 0 1]
Public Key (a, b): (array([3, 1, 4]), array([6, 0, 4]))
------------------------------
Ciphertext c1: [1 0 4]
Ciphertext c2: [6 3 0]
------------------------------
Decrypted Message: [0 1 1]


In [ ]:
# In this easy example accuracy can't be 100 percent
# And if we will do bigger n, accuracy will decrese

repeat_count = 10000
success_count = 0
i = repeat_count
while i: 
    q_conf = 7
    n_conf = 3
    m_conf = np.random.randint(0, 2, size=n_conf)  # Binary message
    priv_key, pub_key, error = keygen(q_conf, n_conf)
    c1, c2 = encrypt(m_conf, pub_key, n_conf, q_conf)
    m = decrypt((c1, c2), priv_key, n_conf, q_conf)

    for j in range(n_conf-1):
        if(m[j] != m_conf[j]):
            success_count -= 1
            break
    
    success_count += 1
    i -= 1

print(f"Accuracy: {round((success_count/repeat_count)*100, 2)}%")


KeyboardInterrupt: 